In [3]:
from selenium import webdriver
import time
import pandas as pd

In [4]:
driver = webdriver.Chrome('./chromedriver')
driver.maximize_window()
driver.implicitly_wait(10)

In [114]:
driver.get('https://basketball.realgm.com/international/league/4/Spanish-ACB/team/649/Acunsa-GBC')
driver.find_element_by_class_name('Frame-sc-1d4hofp-0.czVBsw').click()

In [31]:
#driver.delete_all_cookies()

In [91]:
uno = driver.find_elements_by_class_name('has-subnav.with-js')
dos = uno[5].click()

In [92]:
driver.find_element_by_class_name('Frame-sc-1d4hofp-0.czVBsw').click()
driver.find_element_by_xpath('//*[@id="site-takeover"]/div[3]/div/div[1]/div[5]/select').click()
driver.find_element_by_xpath('//*[@id="site-takeover"]/div[3]/div/div[1]/div[5]/select/option[6]').click()
driver.find_element_by_class_name('Frame-sc-1d4hofp-0.czVBsw').click()


### With this cell we get the name of each stat column.

In [62]:
tablita = driver.find_element_by_class_name("tablesaw.compact.tablesaw-swipe")
columnas = tablita.find_elements_by_tag_name("th")
interm = []
for a in columnas:
    interm.append(a)
columns_name = []
for name in interm: 
    columns_name.append(name.text)

### With this cell we get all the stats of every player in this season (2020-21).

In [116]:
busqueda = driver.find_element_by_id("teamnav")
equipos = busqueda.find_elements_by_class_name("has-subnav")


In [117]:
len(equipos)

19

In [118]:
stats_list = []
for equipo in equipos:
    equipo.click()
    #clicko en la cookie
    driver.find_element_by_class_name('Frame-sc-1d4hofp-0.czVBsw').click()
    uno = driver.find_elements_by_class_name('has-subnav.with-js')
    dos = uno[5].click()
    driver.find_element_by_xpath('//*[@id="site-takeover"]/div[3]/div/div[1]/div[5]/select').click()
    driver.find_element_by_xpath('//*[@id="site-takeover"]/div[3]/div/div[1]/div[5]/select/option[6]').click()
    driver.find_element_by_class_name('Frame-sc-1d4hofp-0.czVBsw').click()
    tablita = driver.find_element_by_class_name("tablesaw.compact.tablesaw-swipe")
    players = tablita.find_elements_by_tag_name("tr")
    for player in players:
        stats = player.find_elements_by_tag_name("td")
        for stat in stats:
            stats_list.append(stat.text)


StaleElementReferenceException: Message: stale element reference: element is not attached to the page document
  (Session info: chrome=90.0.4430.72)
